# Análisis de Brechas de Género - UdeC
## Limpieza de Datos y Generación de Base Maestra

In [1]:
# CELDA 1: LIMPIEZA DE DATOS
import pandas as pd
import numpy as np

# Leer archivos
cuestionario = pd.read_csv('data/cuestionario_original.csv', sep=',', encoding='utf-8')
egresos = pd.read_csv('data/egresados_original.csv', sep=';', encoding='latin-1')
ingresos = pd.read_csv('data/ingresos_original.csv', sep=';', encoding='utf-8', skiprows=3)

# Diccionario de mapeo
diccionario = {3309: 'Ingeniería Civil Industrial', 3310: 'Ingeniería Civil',
    3311: 'Ingeniería Civil Eléctrica', 3318: 'Ingeniería Civil Electrónica',
    3319: 'Ingeniería Civil Informática', 13072.0: 'Ingeniería Civil Industrial',
    13069.0: 'Ingeniería Civil', 13070.0: 'Ingeniería Civil Eléctrica',
    13071.0: 'Ingeniería Civil Electrónica', 13073.0: 'Ingeniería Civil Informática',
    'Masculino': 'H', 'Femenino': 'M', 'MASCULINO': 'H', 'FEMENINO': 'M',
    'INGENIERIA CIVIL INDUSTRIAL': 'Ingeniería Civil Industrial',
    'INGENIERIA CIVIL': 'Ingeniería Civil',
    'INGENIERIA CIVIL ELECTRICA': 'Ingeniería Civil Eléctrica',
    'INGENIERIA CIVIL ELECTRONICA': 'Ingeniería Civil Electrónica',
    'INGENIERIA CIVIL INFORMATICA': 'Ingeniería Civil Informática'}

carreras_validas = ['Ingeniería Civil Industrial', 'Ingeniería Civil',
    'Ingeniería Civil Eléctrica', 'Ingeniería Civil Electrónica', 'Ingeniería Civil Informática']

def filtrar_carreras(df, col): return df[df[col].isin(carreras_validas)]
def filtrar_genero(df, col): return df[df[col].isin(['H', 'M'])]
def clasificar_riesgo(x): return 'Bajo' if x == 1 else ('Medio' if x in [2,3] else ('Alto' if x in [4,5] else None))

# Limpiar cuestionario
cuestionario = cuestionario.iloc[:, np.r_[0, 2:16, 71]]
cuestionario.columns = ['carrera', 'año_ingreso', 'genero', 'reprobaciones', 'asistencia',
    'motivacion', 'participacion', 'autoconfianza_1', 'autoconfianza_2', 'autoconfianza_3',
    'autoconfianza_4', 'autoconfianza_5', 'autoconfianza_6', 'autoconfianza_7',
    'autoconfianza_8', 'intencion_abandono']
cuestionario['autoconfianza'] = (cuestionario.iloc[:, 7:15].mean(axis=1) + 0.5).astype(int)
cuestionario = cuestionario.drop(columns=cuestionario.columns[7:15]).replace(diccionario)
cuestionario = cuestionario[cuestionario['año_ingreso'] != 'Antes de 2015']
cuestionario['año_ingreso'] = cuestionario['año_ingreso'].astype(int)
cuestionario = filtrar_carreras(filtrar_genero(cuestionario, 'genero'), 'carrera')
cuestionario['riesgo'] = cuestionario['intencion_abandono'].apply(clasificar_riesgo)

# Limpiar ingresos
ingresos = ingresos.iloc[:, np.r_[1:4, 9]]
ingresos.columns = ['carrera', 'año_ingreso', 'genero', 'puntaje_ingreso']
ingresos = filtrar_carreras(filtrar_genero(ingresos.replace(diccionario), 'genero'), 'carrera')
ingresos['puntaje_ingreso'] = ingresos['puntaje_ingreso'].astype(str).str.replace(',', '.').astype(float).astype(int)
ingresos['año_ingreso'] = ingresos['año_ingreso'].astype(int)

# Limpiar egresos
egresos = egresos[egresos['NOMBRE INSTITUCIÓN'] == 'UNIVERSIDAD DE CONCEPCION'].iloc[:, np.r_[0,1,2,3,14]]
egresos.columns = ['año_egreso', 'total_titulaciones', 'titulaciones_mujeres', 'titulaciones_hombres', 'carrera']
egresos = filtrar_carreras(egresos.replace(diccionario), 'carrera')
egresos['año_egreso'] = egresos['año_egreso'].astype(str).str.replace('TIT_', '').astype(int)
egresos['titulaciones_mujeres'] = egresos['titulaciones_mujeres'].astype(float).round().astype('Int64').fillna(0)
egresos['titulaciones_hombres'] = egresos['titulaciones_hombres'].astype(float).round().astype('Int64').fillna(0)

print(f'Limpieza completada: {len(cuestionario)} cuestionarios, {len(ingresos)} ingresos, {len(egresos)} egresos')

Limpieza completada: 1283 cuestionarios, 1535 ingresos, 115 egresos


In [2]:
# CELDA 2: CREACIÓN DE BASE MAESTRA

# Agregar ingresos
ing_agr = ingresos.groupby(['año_ingreso','carrera','genero']).agg({'puntaje_ingreso':['mean','count']}).reset_index()
ing_agr.columns = ['año','carrera','genero','puntaje_promedio','cantidad_ingresos']
ing_agr['puntaje_promedio'] = ing_agr['puntaje_promedio'].round(2)
ing_piv = ing_agr.pivot_table(index=['año','carrera'], columns='genero', values=['cantidad_ingresos','puntaje_promedio']).reset_index()
ing_piv.columns = ['año','carrera','ingresos_H','ingresos_M','puntaje_H','puntaje_M']
ing_piv['ingresos_H'] = ing_piv['ingresos_H'].fillna(0).astype(int)
ing_piv['ingresos_M'] = ing_piv['ingresos_M'].fillna(0).astype(int)
ing_piv.loc[ing_piv['ingresos_M']==0, 'puntaje_M'] = np.nan
ing_piv.loc[ing_piv['ingresos_H']==0, 'puntaje_H'] = np.nan
ing_piv['total_ingresos'] = ing_piv['ingresos_M'] + ing_piv['ingresos_H']
ing_piv['brecha_ingresos'] = ing_piv['ingresos_M'] - ing_piv['ingresos_H']
ing_piv['brecha_puntaje'] = (ing_piv['puntaje_M'] - ing_piv['puntaje_H']).round(2)

# Agregar egresos
egr_agr = egresos.groupby(['año_egreso','carrera']).agg({'titulaciones_mujeres':'sum','titulaciones_hombres':'sum','total_titulaciones':'sum'}).reset_index()
egr_agr['brecha_titulaciones'] = egr_agr['titulaciones_mujeres'] - egr_agr['titulaciones_hombres']
egr_piv = egr_agr.rename(columns={'año_egreso':'año','titulaciones_hombres':'titulaciones_H','titulaciones_mujeres':'titulaciones_M'})

# Agregar factores
fac_agr = cuestionario.groupby(['año_ingreso','carrera','genero']).agg({'motivacion':'mean','asistencia':'mean','participacion':'mean','autoconfianza':'mean'}).reset_index()
fac_piv = fac_agr.pivot_table(index=['año_ingreso','carrera'], columns='genero', values=['motivacion','asistencia','participacion','autoconfianza']).reset_index()
fac_piv.columns = ['año','carrera','asistencia_H','asistencia_M','autoconfianza_H','autoconfianza_M','motivacion_H','motivacion_M','participacion_H','participacion_M']
for col in ['asistencia_H','asistencia_M','autoconfianza_H','autoconfianza_M','motivacion_H','motivacion_M','participacion_H','participacion_M']:
    fac_piv[col] = fac_piv[col].round(2)
fac_piv['brecha_motivacion'] = (fac_piv['motivacion_M'] - fac_piv['motivacion_H']).round(2)
fac_piv['brecha_asistencia'] = (fac_piv['asistencia_M'] - fac_piv['asistencia_H']).round(2)
fac_piv['brecha_participacion'] = (fac_piv['participacion_M'] - fac_piv['participacion_H']).round(2)
fac_piv['brecha_autoconfianza'] = (fac_piv['autoconfianza_M'] - fac_piv['autoconfianza_H']).round(2)

# Agregar reprobaciones
rep_agr = cuestionario.groupby(['año_ingreso','carrera','genero']).agg({'reprobaciones':['mean','min','max']}).reset_index()
rep_agr.columns = ['año','carrera','genero','reprobaciones_promedio','reprobaciones_min','reprobaciones_max']
rep_piv = rep_agr.pivot_table(index=['año','carrera'], columns='genero', values=['reprobaciones_promedio','reprobaciones_min','reprobaciones_max']).reset_index()
rep_piv.columns = ['año','carrera','repr_prom_H','repr_prom_M','repr_min_H','repr_min_M','repr_max_H','repr_max_M']
for col in ['repr_prom_H','repr_prom_M','repr_min_H','repr_min_M','repr_max_H','repr_max_M']:
    rep_piv[col] = rep_piv[col].round(2)
rep_piv['brecha_reprobaciones'] = (rep_piv['repr_prom_M'] - rep_piv['repr_prom_H']).round(2)

# Agregar riesgo
rie_agr = cuestionario.groupby(['año_ingreso','carrera','genero','riesgo']).size().reset_index(name='count')
rie_piv = rie_agr.pivot_table(index=['año_ingreso','carrera','genero'], columns='riesgo', values='count', fill_value=0).reset_index()
if 'Bajo' in rie_piv.columns and 'Medio' in rie_piv.columns and 'Alto' in rie_piv.columns:
    total = rie_piv['Bajo'] + rie_piv['Medio'] + rie_piv['Alto']
    rie_piv['bajo_pct'] = (rie_piv['Bajo']/total*100).round(2)
    rie_piv['medio_pct'] = (rie_piv['Medio']/total*100).round(2)
    rie_piv['alto_pct'] = (rie_piv['Alto']/total*100).round(2)
rie_fin = rie_piv.pivot_table(index=['año_ingreso','carrera'], columns='genero', values=['bajo_pct','medio_pct','alto_pct']).reset_index()
rie_fin.columns = ['año','carrera','riesgo_alto_H','riesgo_alto_M','riesgo_bajo_H','riesgo_bajo_M','riesgo_medio_H','riesgo_medio_M']

# Factores por nivel de riesgo
fac_rie = cuestionario.groupby(['año_ingreso','carrera','genero','riesgo']).agg({'motivacion':'mean','asistencia':'mean','participacion':'mean','autoconfianza':'mean','reprobaciones':['mean','min','max']}).reset_index()
fac_rie.columns = ['año','carrera','genero','riesgo','motivacion','asistencia','participacion','autoconfianza','repr_prom','repr_min','repr_max']
fac_rie_piv = fac_rie.pivot_table(index=['año','carrera','riesgo'], columns='genero', values=['motivacion','asistencia','participacion','autoconfianza','repr_prom','repr_min','repr_max']).reset_index()
fac_rie_piv.columns = ['año','carrera','riesgo','asistencia_H','asistencia_M','autoconfianza_H','autoconfianza_M','motivacion_H','motivacion_M','participacion_H','participacion_M','repr_prom_H','repr_prom_M','repr_min_H','repr_min_M','repr_max_H','repr_max_M']
for col in ['asistencia_H','asistencia_M','autoconfianza_H','autoconfianza_M','motivacion_H','motivacion_M','participacion_H','participacion_M','repr_prom_H','repr_prom_M','repr_min_H','repr_min_M','repr_max_H','repr_max_M']:
    fac_rie_piv[col] = fac_rie_piv[col].round(2)

fac_rie_bajo = fac_rie_piv[fac_rie_piv['riesgo']=='Bajo'].drop(columns=['riesgo'])
fac_rie_bajo.columns = ['año','carrera'] + [f'{col}_bajo' for col in fac_rie_bajo.columns[2:]]
fac_rie_medio = fac_rie_piv[fac_rie_piv['riesgo']=='Medio'].drop(columns=['riesgo'])
fac_rie_medio.columns = ['año','carrera'] + [f'{col}_medio' for col in fac_rie_medio.columns[2:]]
fac_rie_alto = fac_rie_piv[fac_rie_piv['riesgo']=='Alto'].drop(columns=['riesgo'])
fac_rie_alto.columns = ['año','carrera'] + [f'{col}_alto' for col in fac_rie_alto.columns[2:]]

# Crear combinaciones y base maestra
años = sorted(set(ing_piv['año'].tolist() + egr_piv['año'].tolist() + cuestionario['año_ingreso'].tolist()))
carreras = sorted(set(ingresos['carrera'].tolist() + egresos['carrera'].tolist() + cuestionario['carrera'].tolist()))
base_maestra = pd.DataFrame([{'año':a,'carrera':c} for a in años for c in carreras])
base_maestra = base_maestra.merge(ing_piv, on=['año','carrera'], how='outer')
base_maestra = base_maestra.merge(egr_piv, on=['año','carrera'], how='outer')
base_maestra = base_maestra.merge(fac_piv, on=['año','carrera'], how='outer')
base_maestra = base_maestra.merge(rep_piv, on=['año','carrera'], how='outer')
base_maestra = base_maestra.merge(rie_fin, on=['año','carrera'], how='outer')
base_maestra = base_maestra.merge(fac_rie_bajo, on=['año','carrera'], how='outer')
base_maestra = base_maestra.merge(fac_rie_medio, on=['año','carrera'], how='outer')
base_maestra = base_maestra.merge(fac_rie_alto, on=['año','carrera'], how='outer')

# Poner NaN donde no hay datos originales
cuest_set = set(zip(cuestionario['año_ingreso'], cuestionario['carrera']))
ingre_set = set(zip(ingresos['año_ingreso'], ingresos['carrera']))
egres_set = set(zip(egresos['año_egreso'], egresos['carrera']))

def poner_nan(fila):
    ac = (fila['año'], fila['carrera'])
    if ac not in cuest_set:
        cols = ['motivacion_M','motivacion_H','asistencia_M','asistencia_H','participacion_M','participacion_H',
                'autoconfianza_M','autoconfianza_H','repr_prom_M','repr_prom_H','repr_min_M','repr_min_H',
                'repr_max_M','repr_max_H','brecha_reprobaciones','riesgo_alto_M','riesgo_alto_H',
                'riesgo_bajo_M','riesgo_bajo_H','riesgo_medio_M','riesgo_medio_H','brecha_motivacion',
                'brecha_asistencia','brecha_participacion','brecha_autoconfianza']
        for niv in ['bajo','medio','alto']:
            for fac in ['asistencia','autoconfianza','motivacion','participacion','repr_prom','repr_min','repr_max']:
                cols.extend([f'{fac}_H_{niv}',f'{fac}_M_{niv}'])
        for col in cols:
            if col in fila.index: fila[col] = np.nan
    if ac not in ingre_set:
        for col in ['puntaje_M','puntaje_H','ingresos_M','ingresos_H','brecha_ingresos','brecha_puntaje','total_ingresos']:
            if col in fila.index: fila[col] = np.nan
    if ac not in egres_set:
        for col in ['titulaciones_M','titulaciones_H','total_titulaciones','brecha_titulaciones']:
            if col in fila.index: fila[col] = np.nan
    return fila

base_maestra = base_maestra.apply(poner_nan, axis=1).sort_values(['carrera','año']).reset_index(drop=True)
base_maestra.to_csv('data/base_maestra.csv', index=False)
print(f'Base maestra: {base_maestra.shape} - {len(carreras)} carreras, {len(años)} años ({min(años)}-{max(años)})')

Base maestra: (95, 80) - 5 carreras, 19 años (2007-2025)
